# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key
#api_key = weather_api_key
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)
print(lat_lngs)
# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)
#debugging no more than 60 per min
cities=cities[0:50]
cities

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
# Perform a weather check on each city using a series of successive API calls.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "Imperial"

weather_url = f"{url}appid={weather_api_key}&units={units}"
# weather_url


In [ ]:
city_data_list = []

#number
number=0
# series_number = 1
# Include a print log of each city as it'sbeing processed (with the city number and city name).
for number, city in enumerate(cities):
    city_dict = {}
    try:
        query_url = f'{weather_url}&q={city}'
        
        response = requests.get(query_url).json()
#         print(response)
        city_dict['city'] = city
        city_dict['temp_max']= response['main']['temp_max']
        city_dict['humidity']= response['main']['humidity']
        city_dict['clouds']= response['clouds']['all']
        city_dict['windspeed']= response['wind']['speed']
        city_dict['lat']=response['coord']['lat']
        city_dict['lng']= response['coord']['lon']
        city_dict['country']= response['sys']['country']
        city_dict['date']= (response['dt'])
        #append to list
        city_data_list.append(city_dict)
#         if number % 50:
#             time.sleep(60)
#             number = 1
#             series_number += 1
#             city_data_list(city_dict)
#         else:
#             number += 1
#             city_data_list(city)
       
        
        print(f'{city} is {number} of {len(cities)}')
        print("---------------------")
    except:
        print('City not found. Skipping...')
        pass
        
print("-----------------------")       
print("Processing Finished!")

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
# Output File (CSV)
output_data_file = "output_data/cities.csv"
#convert to dataframe
weather_data = pd.DataFrame(city_data_list)
weather_data.to_csv('output_data_file')
weather_data.head()

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.
max_humidity = weather_data['humidity'].max()
max_humidity

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
# Build a scatter plot for each data type
plt.scatter(weather_data["lat"], weather_data["temp_max"], marker="o")

# Incorporate the other graph properties
plt.title("Temperature in World Cities")
plt.ylabel("Temperature (Fahrenheit)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("TemperatureInWorldCities.png")

# Show plot
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
# Build a scatter plot for each data type
plt.scatter(weather_data["lat"], weather_data["humidity"], marker="o")

# Incorporate the other graph properties
plt.title("Humidity in World Cities")
plt.ylabel("Humitidy (%)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("HumidityInWorldCities.png")

# Show plot
plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
# Build a scatter plot for each data type
plt.scatter(weather_data["lat"], weather_data["clouds"], marker="o")

# Incorporate the other graph properties
plt.title("cloudiness in World Cities")
plt.ylabel("cloudiness (%)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("CloudinessInWorldCities.png")

# Show plot
plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
# Build a scatter plot for each data type
plt.scatter(weather_data["lat"], weather_data["windspeed"], marker="o")

# Incorporate the other graph properties
plt.title("Windspeed in World Cities")
plt.ylabel("windspeed (mph)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("WindspeedInWorldCities.png")

# Show plot
plt.show()

## Linear Regression

In [ ]:
regress_model = linregress(weather_data["lat"],weather_data["temp_max"] )
regress_model


In [ ]:
x_values=weather_data["lat"] 
#y= mx +b
m= regress_model[0]
b= regress_model[1]
y_values= (m * x_values) + b
# line_eq = "y = "


# plt.scatter(cap_avg["Weight (g)"], cap_avg["Tumor Volume (mm3)"])

# print(plt.scatter)
# plt.plot(x_values, y_values, "r-")
# plt.plot

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
#separate northern and southern hemispheres
northern_hem = weather_data.loc[weather_data["lat"] >= 0]
southern_hem = weather_data.loc[weather_data["lat"] < 0]

#print scatter plot with regression
plt.scatter(northern_hem['lat'],northern_hem['temp_max'])
plt.title("Northern Hemisphere Temperature Regression")
plt.ylabel("Temperature (Fahrenheit)")
plt.xlabel("Latitude")
plt.grid(True)

print(plt.scatter)
plt.plot(x_values, y_values, "r-")
plt.plot


####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
#southern hem scatter and regress
plt.scatter(southern_hem['lat'],southern_hem['temp_max'])
plt.title("Southern Hemisphere Temperature Regression")
plt.ylabel("Temperature (Fahrenheit)")
plt.xlabel("Latitude")
plt.grid(True)

print(plt.scatter)
plt.plot(x_values, y_values, "r-")
plt.plot

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#regress for humidity
regress_model = linregress(weather_data["lat"],weather_data["humidity"] )
x_values=weather_data["lat"] 
#y= mx +b
m= regress_model[0]
b= regress_model[1]
y_values= (m * x_values) + b

#scatter
plt.scatter(northern_hem['lat'],northern_hem['humidity'])
plt.title("Northern Hemisphere Humidity Regression")
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")
plt.grid(True)

# # Print r value
# print(f"The r-value is: {rvalue**2}")

print(plt.scatter)
plt.plot(x_values, y_values, "r-")
plt.plot

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
plt.scatter(southern_hem['lat'],southern_hem['humidity'])
plt.title("Southern Hemisphere Humidity Regression")
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")
plt.grid(True)

print(plt.scatter)
plt.plot(x_values, y_values, "r-")
plt.plot

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
#regress for humidity
regress_model = linregress(weather_data["lat"],weather_data["clouds"] )
x_values=weather_data["lat"] 
#y= mx +b
m= regress_model[0]
b= regress_model[1]
y_values= (m * x_values) + b


plt.scatter(northern_hem['lat'],northern_hem['clouds'])
plt.title("Northern Hemisphere Cloudiness Regression")
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.grid(True)

print(plt.scatter)
plt.plot(x_values, y_values, "r-")
plt.plot

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
plt.scatter(southern_hem['lat'],southern_hem['clouds'])
plt.title("Southern Hemisphere Cloudiness Regression")
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.grid(True)

print(plt.scatter)
plt.plot(x_values, y_values, "r-")
plt.plot

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
#regress for humidity
regress_model = linregress(weather_data["lat"],weather_data["windspeed"] )
x_values=weather_data["lat"] 
#y= mx +b
m= regress_model[0]
b= regress_model[1]
y_values= (m * x_values) + b

plt.scatter(northern_hem['lat'],northern_hem['windspeed'])
plt.title("Northern Hemisphere Windspeed Regression")
plt.ylabel("Windspeed (Mph)")
plt.xlabel("Latitude")
plt.grid(True)

print(plt.scatter)
plt.plot(x_values, y_values, "r-")
plt.plot

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
plt.scatter(southern_hem['lat'],southern_hem['windspeed'])
plt.title("Northern Hemisphere Windspeed Regression")
plt.ylabel("Windspeed (Mph)")
plt.xlabel("Latitude")
plt.grid(True)

print(plt.scatter)
plt.plot(x_values, y_values, "r-")
plt.plot